## Graphs example

This notebook demonstrates some of the functionality of topos-tool in the example of the category of graphs (i.e. directed multigraphs).
We start by defining a basis category.
Note that we call this one "Graphs" but it is the category of presheaves on it that are graphs.
Also note that this particular example can be loaded by typing `open Examples.Graphs` but we make it manually here for demo purposes.

In [2]:
#load "../scripts/LoadCompiled.fsx" // This loads `ToposTool.dll`.
#load "../scripts/LatexPrint.fsx" // This loads the `Latex.print` command.

open LatexPrint

In [3]:
type Graphs = Graphs of string // Define type for the objects of the category. Not strictly necessary but helps with type-safety.


let V, E = Graphs "V", Graphs "E" // Define basis objects representing the vertex and the edge.

let objects = set [ V; E ]


let s, t = Arrow.make "s" V E, Arrow.make "t" V E // Define source and target arrows from the edge to the vertex.

let arrows = set [ s; t ]


let compose = Map.empty // (There are no nontrivial relationships between arrows.)


let cat = Category.make "Graphs" objects arrows compose // Make the category.

cat |> Latex.print

In [4]:
let yo = Yoneda.yo cat // Define the Yoneda functor.

let hV, hE = yo.Object V, yo.Object E // Define the representable presheaves.


hV |> Latex.print // hV is the graph that is a single vertex.

In [5]:
hE |> Latex.print // hE is the graph that is two vertices and one edge between them.

In [6]:
let (*) F G = Presheaf.product F G // Define shortcuts for basic operations.
let (+) F G = Presheaf.sum F G
let (^) F G = Presheaf.exp cat G F


Presheaf.isIso (hE * hE) (hE + (hV + hV)) // We can verify some identities: hE * hE = hE + (hV + hV)

True

In [7]:
hE ^ hV |> Latex.print // hE ^ hV is a graph with two vertices and four edges, two of them loops on the vertices and the other two going from one vertex to the other in opposite directions.

In [8]:
// The generated names can be complicated, but we can check it's isomorphic to the one we described.

// To define a presheaf manually we only need to give the nontrivial data:

let F = 
    let ob = map [V, set ["v_0"; "v_1"]
                  E, set ["e_0"; "e_1"; "e_2"; "e_3"]]

    let ar = map [s, map [("e_0", "v_0"); ("e_1", "v_0"); ("e_2", "v_1"); ("e_3", "v_1")]
                  t, map [("e_0", "v_0"); ("e_1", "v_1"); ("e_2", "v_1"); ("e_3", "v_0")]]

    Presheaf.make "{{h_E}^{h_V}}" cat ob ar

Presheaf.isIso F (hE ^ hV)

True

In [9]:
let subalg = Subobject.subalgebra cat F // Define the algebra of subgraphs of our last example.

let subobjects = subalg.Subobjects


subobjects |> Set.count // There are 21 subgraphs.

21

In [10]:
let omega = Truth.omega cat // Define the truth object Omega of the category, defined by Sub F ~= hom <F, Omega> (an isomorphism of functors).


Morphism.hom F omega |> Set.count // We can at least check for now that as sets they are isomorphic.

21

In [11]:
// The subobjects have the structure of a biheyting algebra that supports operations like meet, join, implication, subtraction, negation, supplement.

let g = subalg.Subobjects |> Seq.item 2 // Take an arbitrary subgraph...


g |> Latex.print // It has two vertices and no edges:

In [12]:
// The negation of a subgraph is the largest subgraph disjoint from it. For g above, that is the empty graph:

let negation = Subobject.negate subalg g


Presheaf.isIso negation (Presheaf.zero cat)

True

In [13]:
// The boundary operator of a subgraph X is the subgraph of vertices of X connected to the outside.

// We will check the product rule for the boundary operator: ∂(x ∧ y) = (∂x ∧ y) ∨ (x ∧ ∂y).

let (+) = Subobject.join // First define some shorthand commands.
let (*) = Subobject.meet
let d = Subobject.boundary subalg

let productRule (X, Y) = d (X * Y) = (d X * Y) + (X * d Y)


subobjects |> Set.square |> Set.forall productRule  // Verify the product rule holds on the subalgebra.

True

In [14]:
// We can glue graphs together using colimits. We will glue the two vertices of hE to make a loop using a coequaliser:

let morphisms = Morphism.hom hV hE // There are two morphisms hV -> hE that send the single vertex of hV to either vertex of hE.


morphisms |> Latex.print

In [15]:
let n = morphisms |> Seq.item 0
let m = morphisms |> Seq.item 1

                                       //                                         n
let L = Presheaf.coequaliser hV n m hE // Take the coequaliser of the diagram hV --> hE .
                                       //                                        --> 
                                       //                                         m

L |> Latex.print // It is a loop: there is one vertex (the set {s,t}) and one edge (the set {1_E}) whose source and target is the vertex.